In [2]:
import matplotlib.pyplot as plt
import jieba.posseg as pseg
import pandas as pd
import numpy as np
import torch.utils.data as Data
import torch.nn as nn
import torch.nn.functional as F
import torch
import jieba
import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Sequential(
            # 卷积参数设置
            nn.Conv2d(
                in_channels=1,  # 输入数据的通道为1，即卷积核通道数为1
                out_channels=16,  # 输出通道数为16，即卷积核个数为16
                kernel_size=(2,300),  # 卷积核的尺寸为 5*5
                stride=1,  # 卷积核的滑动步长为1
                padding=2,  # 边缘零填充为2
            ),
            nn.ReLU(),  # 激活函数为Relu
            nn.MaxPool2d(2),  # 最大池化 2*2
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, (3,300), 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        # 全连接层
        self.fc1 = nn.Sequential(
            nn.Linear(32 * 7 * 7, 120),
            nn.ReLU(),
        )
        self.fc2 = nn.Sequential(
            nn.Linear(120, 36),
            nn.ReLU(),
        )
        self.fc3 = nn.Linear(36, 10)  # 最后输出结果个数为2
        self.soft = nn.Softmax()


    def forward(self,x):
        in_size = x.size(0)
        out = self.relu(self.mp(self.conv1(x)))
        out = self.relu(self.mp(self.conv2(out)))
        out = out.view(in_size, -1)
        out = self.relu(self.fc1(out))
        out = self.relu(self.fc2(out))
        out = self.fc3(out)
        return self.soft(out)

In [4]:
model=CNN()

In [5]:
print('loading all-in-one')
all_in_one=pd.read_csv('/Users/jasondennis/Desktop/graduate-design/data/all-in-one.csv')
print('load positive')
pos=pd.read_csv('/Users/jasondennis/Desktop/graduate-design/data/positive.csv')
print('load negative')
neg=pd.read_csv('/Users/jasondennis/Desktop/graduate-design/data/negative.csv')

loading all-in-one
load positive
load negative


In [6]:
print(all_in_one)

                                                  Comment  posneg
0        机甲之战超超好看，比变形金刚强；人，神，变异人，人工智能互殴，强强强强；每一小段末句都是槽...       1
1        说实话其实剧情就那样吧，非漫威粉看着可能有的地方会get不到G点吧。（其实漫威卖的不是剧情...       1
2                                     没有了洛基这个小基仔真是觉得即墨如雪啊       1
3        看毕，我激动地对友人说，等等奥创要来毁灭台北怎么办厚，她拍了拍我肩膀，没事，反正你买了两份...       1
4                                     哪个保证一刀不剪的，不是剪了好几刀吗！       1
...                                                   ...     ...
739275                   电影很一般，远不如冰河世纪这些，正常打7分，打1分是因为水军太多       0
739276   作为喜剧笑点太生硬而且低龄化，情节走向俗套，感觉不如迪士尼的任何一部动画电影，在我心中和喜...       0
739277                  为什么每次这种莫名其妙的动画片分数这么高，豆瓣这群装逼犯又发作了吗       0
739278                                                 无感       0
739279                                          我来拉分，睡着了都       0

[739280 rows x 2 columns]


In [7]:
sentence=list(all_in_one['Comment'])
seg=list(all_in_one['posneg'])

In [9]:
# sentence_jieba=[ ]
# for x in range(len(all_in_one)):
#     sentence_jieba.append(build_sentence_vec(all_in_one['Comment'][x],300,w2v_model))
#     score.append(all_in_one['posneg'][x])
# print('done')

In [10]:
import jieba.posseg as pseg

sentence_word=[ ]
for i in range(len(sentence)):

    list=[ ]
    tag_filter = ['n', 'v','s','a','z','y']  # 需要保留的词性
    seg_result = pseg.cut(sentence[i])  # 结果是一个pair，有flag和word两种值
    list.append(s.word for s in seg_result if s.flag in tag_filter)
    sentence_word.append(list)
    
    
print('done')

done


In [10]:
# import jieba.posseg as pseg
# list = []
# tag_filter = ['n', 'v','s','a','z','y']  # 需要保留的词性
# seg_result = pseg.cut(sentence[1])  # 结果是一个pair，有flag和word两种值
# list.append([" ".join(s.word for s in seg_result if s.flag in tag_filter)])
# print("词性过滤完成")
# print(list)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/y_/v6f38kjs6tncrsfkklcmjf640000gn/T/jieba.cache
Loading model cost 0.744 seconds.
Prefix dict has been built successfully.


词性过滤完成
[['剧情 吧 看着 可能 有 地方 会 不到 吧 剧情 人物 呀 宇宙 棒棒 漫威粉 全程 高能 燃爆 大法 好']]


In [8]:
w2v_model=KeyedVectors.load_word2vec_format("/Users/jasondennis/Desktop/graduate-design/CNN/test1/sgns.zhihu.bigram.bz2",binary=False)
print('done')

done


In [12]:
import torch
import numpy as np
import math

def build_sentence_vec(sentence_word,asize,bsize,w2v_model):
    x=torch.empty(1,bsize)
    y=torch.zeros(1,bsize)
#     for i in range(len(sentence_word)):
#         gap=sentence_word[i]
        if asize<=len(sentence_word):
            for j in range(asize):
                x=torch.cat((x,torch.from_numpy(w2v_model[gap[j]].reshape((1,size)))),0)
        else:
            for j in range(len(sentence_word)):
                x=torch.cat((x,torch.from_numpy(w2v_model[gap[j]].reshape((1,size)))),0)
            for j in range(asize,len(sentence_word),1):
                x=torch.cat((x,y),0)
    return x

        
            

In [15]:
print(build_sentence_vec(sentence_word[1],36,300,w2v_model))

TypeError: 'generator' object is not subscriptable